In [1]:
%pip install transformers==4.2.1

Note: you may need to restart the kernel to use updated packages.


In [30]:
import numpy as np
import pandas as pd
import re
import os
import sys

sys.path.append('../../Quantlet/Create_description/')

import importlib
import preprocessing_utils
importlib.reload(preprocessing_utils)

import pickle

from sklearn.model_selection import train_test_split, KFold

In [31]:
RS = 42

In [32]:
parsed_Qs_file = '../../data/preprocessed/Quantlet/Parsed_Qs_19032023.pkl'
dataset = preprocessing_utils.QuantletDataset(parsed_Qs_file)

dataset = dataset.parsed_Qs_file
dataset.type_script = dataset.type_script.str.replace('ipynb', 'py')
dataset['most_commen_lang'] = dataset.folder_name.map(dataset.groupby(['folder_name'])['type_script'].agg(pd.Series.mode))
dataset['most_commen_lang'] = dataset['most_commen_lang'].astype(str)
dataset['desc_idx'] = dataset.index

In [33]:
tocheck_data = dataset.copy()
tocheck_data.metainfo_file = tocheck_data.metainfo_file.astype(str)
tocheck_data['empty'] = (tocheck_data.metainfo_file=='empty').astype(int)
tocheck_data['empty_ratio'] = tocheck_data.folder_name.map(tocheck_data.groupby('folder_name').empty.sum()) / tocheck_data.folder_name.map(tocheck_data.groupby('folder_name').empty.count())

q_no_meta = tocheck_data[tocheck_data.empty_ratio==1].folder_name.unique()

In [34]:
Q_lang = dataset[['folder_name', 'most_commen_lang']].drop_duplicates().reset_index(drop=True)
Q_lang = Q_lang[~Q_lang.folder_name.isin(q_no_meta)]

In [35]:
labelled, test = train_test_split(Q_lang, test_size=0.2, random_state=RS, stratify=Q_lang.most_commen_lang)
print(labelled.shape, test.shape)

(2376, 2) (595, 2)


In [39]:
no_meta_ds = dataset.query('folder_name in @q_no_meta')
labelled_ds = dataset.query('folder_name in @labelled.folder_name')
unlabelled_ds = dataset.query('folder_name in @test.folder_name')
print(labelled_ds.shape, unlabelled_ds.shape)
full = labelled_ds.shape[0] + unlabelled_ds.shape[0]
print(labelled_ds.shape[0] / full, unlabelled_ds.shape[0]/full)

(3871, 7) (968, 7)
0.7999586691465179 0.20004133085348214


In [41]:
with open('../../data/preprocessed/Quantlet/Descriptions_Qs_19032023.pkl', 'rb') as f:
    descriptions = pickle.load(f)

In [ ]:

from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
